# **Aplicaciones Clásicas de IA en Visión por Computadora**
**Instalación de dependencias**:
```bash
!pip install datasets scikit-learn scikit-image opencv-python matplotlib numpy
```

In [1]:
from datasets import load_dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## **1. Cargar Dataset (CIFAR-10)**

In [5]:
# Cargar dataset CIFAR-10 desde Hugging Face
dataset = load_dataset("cifar10")

# Convertir a numpy arrays
def process_dataset(batch):
    images = [np.array(img.convert('RGB')) for img in batch["img"]]  # Ensure RGB format
    return {
        "image": [img.astype(np.uint8) for img in images],  # Force uint8 conversion
        "label": batch["label"]
    }


dataset = dataset.map(process_dataset, batched=True)
train_images = np.array(dataset["train"]["image"])
train_labels = np.array(dataset["train"]["label"])
test_images = np.array(dataset["test"]["image"])
test_labels = np.array(dataset["test"]["label"])

## **2. Extracción de Características**

In [2]:
def extract_features(images, radius=3, n_points=24, pca_components=28):
    features = []

    for img in images:
        # Ensure correct dtype and color space
        img = np.array(img)
        if img.dtype != np.uint8:
            img = img.astype(np.uint8)

        # Convert to grayscale properly
        if len(img.shape) == 3:  # If color image (H, W, C)
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        else:  # Already grayscale
            gray = img.copy()

        # 1. Normalización CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        normalized = clahe.apply(gray)

        # 2. LBP
        lbp = local_binary_pattern(normalized, n_points, radius, method='uniform')
        hist_lbp, _ = np.histogram(lbp, bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

        # 3. Haralick
        glcm = graycomatrix(normalized, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast')[0, 0]
        energy = graycoprops(glcm, 'energy')[0, 0]

        # Combinar características
        features.append(np.concatenate([hist_lbp, [contrast, energy]]))

    # Aplicar PCA
    pca = PCA(n_components=pca_components)
    return pca.fit_transform(np.array(features))

## **3. Entrenamiento y Evaluación**

In [ ]:
# Crear y entrenar MLP

# 4. Extraer características (¡Ejecutar esto primero!)
X_train = extract_features(train_images)
X_test = extract_features(test_images)
y_train = train_labels
y_test = test_labels

In [ ]:

mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    max_iter=500,
    alpha=0.001,
    learning_rate_init=0.001,
    verbose=True
)

mlp.fit(X_train, y_train)

# Evaluar
preds = mlp.predict(X_test)
print(classification_report(y_test, preds))

## **4. Ejemplo con Dataset Médico (skin cancer)**

In [3]:
dataset = load_dataset("marmal88/skin_cancer")  # 10,015 imágenes

def process_medical_images(batch):
    processed = []
    for img in batch["image"]:
        # Preprocesamiento específico para imágenes médicas
        img = np.array(img)
        gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        resized = cv2.resize(gray, (128, 128))
        processed.append(resized)
    return {"image": processed, "label": batch["dx"]}

skin_dataset = dataset.map(process_medical_images, batched=True)

# Extraer características y entrenar
X_medical = extract_features(skin_dataset["train"]["image"])
y_medical = skin_dataset["train"]["dx"]

X_train_med, X_test_med, y_train_med, y_test_med = train_test_split(X_medical, y_medical, test_size=0.2)

mlp_med = MLPClassifier(hidden_layer_sizes=(50, 25))
mlp_med.fit(X_train_med, y_train_med)
print(classification_report(y_test_med, mlp_med.predict(X_test_med)))

                               precision    recall  f1-score   support

            actinic_keratoses       0.21      0.11      0.15        63
         basal_cell_carcinoma       0.17      0.05      0.07        88
benign_keratosis-like_lesions       0.29      0.24      0.26       203
               dermatofibroma       0.00      0.00      0.00        18
             melanocytic_Nevi       0.80      0.84      0.82      1284
                     melanoma       0.27      0.38      0.31       233
             vascular_lesions       0.00      0.00      0.00        27

                     accuracy                           0.64      1916
                    macro avg       0.25      0.23      0.23      1916
                 weighted avg       0.61      0.64      0.62      1916



## **5. Visualización de Características**

In [4]:
# Mostrar ejemplos
def plot_sample_features(idx):
    img = train_images[idx]
    features = X_train[idx]
    
    plt.figure(figsize=(15,5))
    plt.subplot(131)
    plt.imshow(img)
    plt.title('Imagen Original')
    
    plt.subplot(132)
    plt.bar(range(len(features)), features)
    plt.title('Características Extraídas')
    
    plt.subplot(133)
    plt.scatter(features[:10], features[10:20])
    plt.title('Espacio de Características')
    plt.show()

plot_sample_features(0)
plot_sample_features(100)

NameError: name 'train_images' is not defined

## **6. Optimización de Hiperparámetros**

In [ ]:
from sklearn.model_selection import GridSearchCV

# Configurar parámetros
parameters = {
    'hidden_layer_sizes': [(50,), (100,), (50, 25)],
    'activation': ['tanh', 'relu'],
    'alpha': [0.0001, 0.001, 0.01]
}

# Búsqueda en grid
clf = GridSearchCV(MLPClassifier(max_iter=200), parameters, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

print(f"Mejores parámetros: {clf.best_params_}")
print(f"Mejor puntuación: {clf.best_score_:.2f}")

## **Referencias**
- CIFAR-10: https://huggingface.co/datasets/cifar10
- Scikit-learn: https://scikit-learn.org
- OpenCV: https://opencv.org